In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

# import geopandas as gpd
# import fiona

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pwd

'/Users/michaelmainzer/Desktop/projects/covid-recovery/data'

In [39]:
# all codes need to be brought in as strings
msa  =  pd.read_excel('../../data/geographies/cbsa.xlsx',
                      sheet_name='cbsa',
                      usecols="A,D,M",
                      converters={'area_code':str,'CBSA Code':str})
rc = pd.read_excel('../../data/geographies/regional-commissions.xlsx',
                   sheet_name='county-rc',
                   usecols="A,C,D",
                   converters={"area_code":str})

In [40]:
msa.sort_values('area_code', ascending='False', inplace=True)
rc.sort_values('area_code', ascending='False', inplace=True)

In [41]:
# get clean covid data
covid = pd.read_csv('../application/app-data/covid-clean.csv')
covid = covid[['date','area_name','area_code','cases','deaths']]

covid.date = pd.to_datetime(covid.date)

covid['area_code'] = covid['area_code'].astype(int)
covid['area_code'] = covid['area_code'].astype(str)

In [ ]:
# get new data from nyt
covidCounty = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
covidCounty = covidCounty.loc[covidCounty['state'] == 'Georgia']

covidCounty = covidCounty[['date','county','fips','cases','deaths']]

# get state level data
covidState = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv')
covidState = covidState.loc[covidState['state'] == 'Georgia']

covidCounty.rename(columns={'county' : 'area_name',
                      'fips'   : 'area_code'}, inplace=True)

covidState.rename(columns={'state' : 'area_name',
                           'fips'  : 'area_code'}, inplace=True)

covid = covidState.append(covidCounty)

# the covid data is reported as a cumulative sum
# we want the daily counts as well for different charting options
covid.date = pd.to_datetime(covid.date)
covid.sort_values(['area_name', 'date'], ascending=[True, True], inplace=True)



covid.dropna(subset=['area_code'], inplace=True)

covid['area_code'] = covid['area_code'].astype(int)
covid['area_code'] = covid['area_code'].astype(str)

covid.reset_index(inplace=True, drop=True)

covid.sort_values(['area_name', 'date'], ascending=[True, True], inplace=True)

# filter only CURRENT DAY'S DATA
# append to cleaned sheet

In [42]:

covid['daily_cases'] = np.where(covid['area_code'] == covid['area_code'].shift(), covid.cases - covid.cases.shift(), np.nan)

covid['daily_cases'].fillna(covid['cases'], inplace=True)

covid['daily_deaths'] = np.where(covid['area_code'] == covid['area_code'].shift(), covid.deaths - covid.deaths.shift(), np.nan)
covid['daily_deaths'].fillna(covid['deaths'], inplace=True)

In [43]:
covidMsa = pd.merge(covid,msa, how='left', on='area_code')

In [44]:
covidMsa.dropna(subset=['CBSA Code'], inplace=True)

In [45]:
covidMsa = covidMsa.groupby(['date','CBSA Title','CBSA Code']).agg({'cases'       :'sum',
                                                                    'deaths'      :'sum',
                                                                    'daily_cases' :'sum',
                                                                    'daily_deaths':'sum'}).reset_index()

In [46]:
covidRc = pd.merge(covid,rc, how='left', on='area_code')

In [47]:
covidRc = covidRc.groupby(['date','Regional Commission','RC_ID']).agg({'cases'       :'sum',
                                                                       'deaths'      :'sum',
                                                                       'daily_cases' :'sum',
                                                                       'daily_deaths':'sum'}).reset_index()

In [48]:
# match the column names and order of all frames

# final has to be the following
# date,area_code (state fips, county fips, cbsa, lwfda_no),area_name,cases,
# deaths,daily_cases,daily_deaths,rolling_7_case_avg,rolling_7_death_avg

covidRc.rename(columns={'Regional Commission' : 'area_name',
                        'RC_ID'               : 'area_code'}, inplace=True)

covidMsa.rename(columns={'CBSA Title' : 'area_name',
                         'CBSA Code'  : 'area_code'}, inplace=True)

In [49]:
covidRc.sort_values(['area_code','date'], ascending=['False','False'], inplace=True)
covidMsa.sort_values(['area_code','date'], ascending=['False','False'], inplace=True)
covid.sort_values(['area_code','date'], ascending=['False','False'], inplace=True)

In [50]:
df = covid.append([covidRc, covidMsa])

In [51]:
df.reset_index(drop=True,inplace=True)

In [52]:
df["date"] = pd.to_datetime(df["date"]).dt.strftime('%m-%d-%Y')

In [53]:
df.head(50)

,date,area_name,area_code,cases,deaths,daily_cases,daily_deaths
0,03-02-2020,Georgia,13,2,0,2.0,0.0
1,03-03-2020,Georgia,13,2,0,0.0,0.0
2,03-04-2020,Georgia,13,2,0,0.0,0.0
3,03-05-2020,Georgia,13,2,0,0.0,0.0
4,03-06-2020,Georgia,13,3,0,1.0,0.0
5,03-07-2020,Georgia,13,7,0,4.0,0.0
6,03-08-2020,Georgia,13,11,0,4.0,0.0
7,03-09-2020,Georgia,13,17,0,6.0,0.0
8,03-10-2020,Georgia,13,22,0,5.0,0.0
9,03-11-2020,Georgia,13,31,0,9.0,0.0


In [54]:
df.to_csv('../application/app-data/covid.csv',index=False)

In [ ]:
# add 7 day running average
# add total new cases last two weeks
# add total new cases per 1000
# add daily growth rate over last 14 days
# new cases over the last 7 days
# new cases over the previous 7-day period
# total change current week to previous week
# week to week change per 100k

In [ ]:
covid.head()